## Reference 
- [Rasa Masterclass - episode 8](https://www.youtube.com/watch?v=9POI7LiKH_8)

## Actual development of assistant
- 그동안 어시스턴트 구축과 관련된 이론적 개념에 대해 다루어왔음 
- 이번 강의는 Medicate locator assistant를 실제로 사용하기 위해 필요한 구축 요소에 대해서 다룸
    - 데이터베이스로부터 다양한 healthy facility에 대한 정보를 가져와 이를 사용자의 인풋에 대해 대답할 때 사용하는 backend integration을 수행
    - 사용할 데이터베이스
        - Data.Medicare.gov
        - https://data.medicare.gov/Hospital-Compare/Hospital-General-Information/xubh-q36u
        - 미국의 다양한 health facilitiy에 대한 데이터를 가지고 있으며, facility 종류에 따라 resource id가 다름 
        - API를 통해 데이터를 가져와 사용할 수 있음    

## 목차
1. Update the NLU stories data of medicare locator assistant
2. Implement form action
3. Implement custom action with real backend integrations
4. Enable the assistant to fail gracefully

### 1. Update the NLU stories data of medicare locator assistant
1. 각 intent에 대한 학습데이터 추가 
    - nlu.md 파일
    - rasa에서 학습데이터 제공

2. entity 추출 
    1. location
        - 사용자는 location에 대한 정보를 도시 이름 또는 우편번호를 통해 제공
        - 우편번호는 형태가 일정하다는 특징을 가지고 있으며, 따라서 regex feature를 통해 entity를 추출하면 성능을 높일 수 있음
        - nlu.md 파일에 특정 entity에 대한 regex feature를 추가하면 됨
        - 형태: 
                regex: entity name
                - regular expression 패턴
        <img src="img\rasa29.png" width="200" height="200">
        
    2. facility_type
        - resource code를 통해 데이터베이스에서 특정 종류의 facility에 대한 정보를 가져올 수 있음
        - 사용자들은 resource code가 아닌 hospital, nursing home 등의 실제 단어로 응답함
        - 따라서 사용자가 제공한 인풋에서 추출된 값을 데이터베이스 탐색에 활용하기 위해선 **data normalization**이 필요
            1. nlu.md 파일에서 synonym으로 extracted entity - resource code 맵핑
                - 형태:
                        sysnonym: resource code
                        - 사용자 인풋에서 추출된 entity value
            <img src="img\rasa30.png" width="200" height="300">
         
            2. config.yml 파일에 EntitysynonymMapper 추가
        
        <br>
        - 결과 
            - rasa train nlu -> rasa shell nlu
            
        <img src="img\rasa31.png" width="500" height="500">

### 2. Implement form action
- 데이터베이스에 API call을 하기 위해선 location과 facility type에 대한 정보가 모두 있어야 함
- rasa form action을 사용
    - 특정 행동을 수행하기 전 필요한 정보를 모두 가지고 있는지 확인해줌
    - action.py 파일에서 정의
    
            1. rask_sdk에서 FormAction을 import
            2. form action class는 총 4가지 요소로 이루어져 있음
                1. name: form action 이름 
                2. require_slots: 어떤 slot을 채워야 하는지 정의. 여기서 정의한 slot이 모두 채워질 때까지 계속해서 form action을 수행하게 됨 
                3. slot_mappings
                    - optional
                    - 기본적으로 form action의 slot을 채울 때, 동일한 이름의 slot으로부터 그 slot value를 가져옴
                    - slot value를 entity에서 가져올 수 있으며, 그 entity가 어떤 intent와 함께 나오는 값인지도 특정할 수 있음 
                4. submit
                    - 모든 slot이 채워진 경우 수행하게 되는 행동 
                    - 사용자의 인풋 데이터에서 얻은 정보를 이용하여 facility search를 한 후에 그 결과값 사용자에게 제공 
                    
    - form action을 사용하기 위해 추가 업데이트 해야 하는 것들
        - domain.yml 파일에서 forms 업데이트
        - config.yml 파일에 FormPolicy추가 
        - stories.md 업데이트 

### 3. Implement custom action with real backend integrations  
- facility_search custom action
    - 어시스턴트가 location과 facility type에 대한 정보들을 이용하여 backend integration을 수행한 후 그 결과값을 사용자에게 제공 
    - FindHealthCareAddress가 가장 중요 (body of the custom action)

### 4. Enable the assistant to fail gracefully
- fallback action
    - 어시스턴트가 사용자의 인풋을 인식하지 못하는 등의 오류가 발생했을 때 이를 gracefully 해결하는 것 
    - TwoStageFallbackPolicy를 사용 
        - config.yml 파일에 업데이트 

## 결과
- rasa run actions & rasa shell

<img src="img\rasa32.png" width="900" height="900">